In [109]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [110]:
df = pd.read_csv("bank-full.csv",sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [111]:
df.shape

(45211, 17)

In [112]:
df.y.value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [113]:
X = df.iloc[ : , : -1]
y = df.y
X

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown


In [114]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
X.loc[ :,['job','marital','education','default','housing','loan','contact','month','poutcome']] = \
X.loc[ :,['job','marital','education','default','housing','loan','contact','month','poutcome']].apply(enc.fit_transform)
X_data = X.head()

In [115]:
X_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3


In [116]:
Y = enc.fit_transform(df.loc[:,'y'])

In [117]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [118]:
# from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [119]:
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
y_predict

array(['no', 'no', 'no', ..., 'no', 'yes', 'no'], dtype=object)

In [120]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.8884547331170746


col_0,no,yes
y,,
no,11763,243
yes,1270,288


## After Applying Smote

In [121]:
#!pip install imblearn

In [122]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [123]:
X_train_smote, y_train_smote = smote.fit_sample(X_train.astype(float),y_train)

In [124]:
from collections import Counter
print("Before Smote : ",Counter(y_train))
print("After Smote : ",Counter(y_train_smote))

Before Smote :  Counter({'no': 27916, 'yes': 3731})
After Smote :  Counter({'no': 27916, 'yes': 27916})


In [125]:
 model.fit(X_train_smote,y_train_smote)
y_predict = model.predict(X_test)
print(accuracy_score(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.7871571807726334


col_0,no,yes
y,,
no,9505,2501
yes,386,1172


In [126]:
classifier = LogisticRegression()
classifier.fit(X_train_smote,y_train_smote)

LogisticRegression()

In [127]:
y_pred = classifier.predict(X_train_smote)

In [128]:
y_pred

array(['no', 'no', 'yes', ..., 'no', 'yes', 'yes'], dtype=object)

In [129]:
y_pred_df= pd.DataFrame({'actual': y_train_smote,
                         'predicted_prob': classifier.predict(X_train_smote)})

In [130]:
y_pred_df

,actual,predicted_prob
0,no,no
1,no,no
2,no,yes
3,no,no
4,no,no
...,...,...
55827,yes,no
55828,yes,yes
55829,yes,no
55830,yes,yes


In [131]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_train_smote,y_pred)
print (confusion_matrix)

[[22026  5890]
 [ 6950 20966]]


In [132]:
#Classification report
from sklearn.metrics import classification_report
print(classification_report(y_train_smote,y_pred))

              precision    recall  f1-score   support

          no       0.76      0.79      0.77     27916
         yes       0.78      0.75      0.77     27916

    accuracy                           0.77     55832
   macro avg       0.77      0.77      0.77     55832
weighted avg       0.77      0.77      0.77     55832



In [133]:
test_data = pd.DataFrame({'age':[58],'job':[3],'marital':[1],'education':[2],'default':[0],'balance':[2500],'housing':[1],'loan':[0],'contact':[2],'day':[5],'month':[8],'duration':[200],'campaign':[1],'pdays':[-1],'previous':[0],'poutcome':[3]})
test_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,3,1,2,0,2500,1,0,2,5,8,200,1,-1,0,3


In [134]:
y_preds = classifier.predict(test_data)


In [135]:
y_preds

array(['no'], dtype=object)